Project #6: TL;DR for News Articles.

In [1]:
!pip install transformers
from transformers import pipeline

In [2]:
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [3]:
article = """
Generative AI refers to a category of artificial intelligence (AI)
techniques and models that are designed to create new content.
Unlike traditional AI, which is often used for classification or prediction,
generative AI can produce text, images, audio, and even video.
At the heart of many modern generative AI systems are Large Language Models (LLMs).
These models are trained on vast amounts of data, allowing them to understand
and generate human-like language. The development of the Transformer
architecture in 2017 was a major turning point, enabling models to process
data in parallel and better understand context.
"""

In [5]:
summary = summarizer(article, max_length=40, min_length=10, do_sample=False)

print("ORIGINAL TEXT")
print(article)
print("\nAI SUMMARY (TL;DR)")
print(summary[0]['summary_text'])

ORIGINAL TEXT

Generative AI refers to a category of artificial intelligence (AI) 
techniques and models that are designed to create new content. 
Unlike traditional AI, which is often used for classification or prediction, 
generative AI can produce text, images, audio, and even video. 
At the heart of many modern generative AI systems are Large Language Models (LLMs). 
These models are trained on vast amounts of data, allowing them to understand 
and generate human-like language. The development of the Transformer 
architecture in 2017 was a major turning point, enabling models to process 
data in parallel and better understand context.


AI SUMMARY (TL;DR)
 Generative AI refers to a category of artificial intelligence (AI) that is designed to create new content . Unlike traditional AI, it can produce text, images, audio, and even video
